### BenchMark in Interaction baselines, FIM excluded

In [1]:
import os
import sys
os.chdir('../')
sys.path.append('../')

import torch
from utils.utils import evaluate,train,prepare
from models.baseline_CNN_CNN import GCAModel

In [2]:
hparams = {
    'scale':'demo',
    'name':'baseline-cnn-cnn',
    'batch_size':10,
    'title_size':30,
    'his_size':50,
    'npratio':4,
    'dropout_p':0.2,
    'query_dim':200,
    'embedding_dim':300,
    'filter_num':400,
    'value_dim':16,
    'head_num':16,
    'epochs':5,
    'metrics':'group_auc,ndcg@5,ndcg@10,mean_mrr',
    'device':'cuda:0',
    'attrs': ['title'],
    'save_each_epoch':False,
    'save_step':None,
    'train_embedding':False,
}

device = torch.device(hparams['device']) if torch.cuda.is_available() else torch.device("cpu")

In [3]:
vocab, loader_train, loader_test, loader_validate = prepare(hparams, validate=True)

In [5]:
gcaModel = GCAModel(hparams, vocab=vocab).to(device)

In [6]:
train(gcaModel, hparams, loader_train, loader_test, loader_validate, save=False)

training...
epoch 0 , step 0 , loss: inf: : 3it [00:01,  1.25it/s]saved model of step 0
epoch 0 , step 190 , loss: 1.5705: : 200it [00:14, 13.40it/s]
epoch 1 , step 0 , loss: inf: : 4it [00:00,  8.10it/s]saved model of step 0
epoch 1 , step 190 , loss: 1.4817: : 200it [00:13, 14.31it/s]
epoch 2 , step 0 , loss: inf: : 3it [00:00,  5.96it/s]saved model of step 0
epoch 2 , step 190 , loss: 1.4081: : 200it [00:14, 14.20it/s]
epoch 3 , step 0 , loss: inf: : 3it [00:00,  6.83it/s]saved model of step 0
epoch 3 , step 190 , loss: 1.3359: : 200it [00:13, 14.31it/s]
epoch 4 , step 0 , loss: inf: : 3it [00:00,  8.39it/s]saved model of step 0
epoch 4 , step 190 , loss: 1.2384: : 200it [00:13, 14.35it/s]
0it [00:00, ?it/s]save success!
testing...
1811it [00:26, 68.27it/s]
0it [00:00, ?it/s]evaluation results:{'group_auc': 0.5748, 'ndcg@5': 0.2764, 'ndcg@10': 0.3446, 'mean_mrr': 0.2609}
validating...
7531it [01:45, 71.21it/s]
evaluation results:{'group_auc': 0.7886, 'ndcg@5': 0.5029, 'ndcg@10': 0.5